In [36]:
import spacy
import pandas as pd
from spacy import displacy
import os

In [37]:
# Cargar el archivo CSV
df = pd.read_csv(r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\limpios\escritos_procesados.csv")

# Ver las primeras filas para entender la estructura
print(df.head())

                                           contenido        titulo
0  Clitofon, acusado por Sócrates de haber censur...      Clitofon
1  La correspondencia que sigue comprende trece c...        Cartas
2  ¿Qué es lo justo? Este es el comienzo , y el a...   De lo justo
3  Hé aquí otra serie de preguntas y respuestas d...  De la virtud
4  La vida no es más que un viaje; y es natural, ...        Axioco


In [38]:
# Carga modelo español large
nlp = spacy.load('es_core_news_lg')

In [39]:
# Ejemplo de un escrito
ejemplo_escrito = df["contenido"][2]  # Tercer escrito
doc_escrito = nlp(ejemplo_escrito)

In [40]:
# Visualización de dependencias
displacy.render(doc_escrito, style="dep", jupyter=True)

In [47]:
# Limpieza básica del texto

def limpiar_texto(texto):
    texto = texto.lower()
    texto = texto.replace('\n', ' ').replace('\r', ' ')
    texto = texto.strip()
    return texto

df["contenido_limpio"] = df["contenido"].apply(limpiar_texto)

In [48]:
# Extracción de preguntas filosóficas 

def extraer_preguntas(texto):
    doc = nlp(texto)
    preguntas = [sent.text for sent in doc.sents if "?" in sent.text or "¿" in sent.text]
    return preguntas

df["preguntas_filosoficas"] = df["contenido_limpio"].apply(extraer_preguntas)

In [49]:
# Extracción de Entidades (NER) antes de procesar el texto

def extraer_entidades(texto):
    doc = nlp(texto)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    return entidades

df["conceptos_clave"] = df["contenido_limpio"].apply(extraer_entidades)

In [50]:
# Procesamiento de Texto (Lematización y eliminación de stopwords)

def procesar_escrito(texto):
    doc = nlp(texto)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

df["contenido_procesado"] = df["contenido_limpio"].apply(procesar_escrito)

In [51]:
# Filtrado de Fragmentos Estilo Socrático

def extraer_fragmentos_socraticos(texto):
    lineas = texto.split("\n")
    fragmentos = [linea for linea in lineas if "¿" in linea or ":" in linea]
    return " ".join(fragmentos)

df["fragmentos_socraticos"] = df["contenido"].apply(extraer_fragmentos_socraticos)

In [52]:
# Resultado final

print(df.head())

                                           contenido        titulo  \
0  Clitofon, acusado por Sócrates de haber censur...      Clitofon   
1  La correspondencia que sigue comprende trece c...        Cartas   
2  ¿Qué es lo justo? Este es el comienzo , y el a...   De lo justo   
3  Hé aquí otra serie de preguntas y respuestas d...  De la virtud   
4  La vida no es más que un viaje; y es natural, ...        Axioco   

                                 contenido_procesado  \
0  clitofon acusado sócrat censurar conversación ...   
1  correspondencia comprender trece carta dirigid...   
2  justo comienzo asunto diálogo injusto ignoranc...   
3  hé serie pregunta respuesta interés literario ...   
4  vida viaje natural necesario término término m...   

                               preguntas_filosoficas  \
0                                                 []   
1                                                 []   
2  [¿qué es lo justo?, ,¿qué demonio ha inspirado...   
3  [hé aquí otra s

In [56]:
df.drop(["contenido", "contenido_limpio", "conceptos_clave"], axis=1, inplace=True)

In [57]:
df.head(5)

,titulo,contenido_procesado,preguntas_filosoficas,fragmentos_socraticos
0,Clitofon,clitofon acusado sócrat censurar conversación ...,[],
1,Cartas,correspondencia comprender trece carta dirigid...,[],La correspondencia que sigue comprende trece c...
2,De lo justo,justo comienzo asunto diálogo injusto ignoranc...,"[¿qué es lo justo?, ,¿qué demonio ha inspirado...","¿Qué es lo justo? Este es el comienzo , y el a..."
3,De la virtud,hé serie pregunta respuesta interés literario ...,[hé aquí otra serie de preguntas y respuestas ...,Hé aquí otra serie de preguntas y respuestas d...
4,Axioco,vida viaje natural necesario término término m...,"[morir es hacerse insensible, es decir, incapa...","La vida no es más que un viaje; y es natural, ..."


In [58]:
output_directory = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed"
df.to_csv(os.path.join(output_directory, "escritos_socraticos_procesados.csv"), index=False)